[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/buddi-colab/blob/main/buddi_colab.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/buddi
%cd /content/buddi

# !wget https://huggingface.co/camenduru/buddi/resolve/main/essentials.zip?download=true -O /content/essentials.zip
# !unzip /content/essentials.zip -d /content/buddi
# !rm -rf /content/buddi/__MACOSX

!wget https://huggingface.co/camenduru/buddi/resolve/main/essentials_new.zip?download=true -O /content/essentials.zip
!unzip /content/essentials.zip -d /content/buddi

!pip install -q https://github.com/camenduru/wheels/releases/download/colab/pytorch3d-0.7.5-cp310-cp310-linux_x86_64.whl
!pip install -q omegaconf pyrender einops loguru wandb smplx

import os
new_path = '/content/buddi'
os.environ['PYTHONPATH'] = f"{new_path}:{os.environ.get('PYTHONPATH', '')}"

!apt -y install -qq aria2
# According to license we have to register and download the models at https://smpl-x.is.tue.mpg.de/register.php
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/buddi/resolve/main/essentials/body_models/smil/smil_web.pkl -d /content/buddi/essentials/body_models/smil -o smil_web.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/buddi/resolve/main/essentials/body_models/smil/smil_packed_info.pth -d /content/buddi/essentials/body_models/smil -o smil_packed_info.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/buddi/resolve/main/essentials/body_models/smil/smplx_kid_template.npy -d /content/buddi/essentials/body_models/smil -o smplx_kid_template.npy
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/buddi/resolve/main/essentials/body_models/smpl/SMPL_NEUTRAL.pkl -d /content/buddi/essentials/body_models/smpl -o SMPL_NEUTRAL.pkl
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/buddi/resolve/main/essentials/body_models/smpl/SMPL_NEUTRAL.pth -d /content/buddi/essentials/body_models/smpl -o SMPL_NEUTRAL.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/buddi/resolve/main/essentials/body_models/smplx/SMPLX_NEUTRAL.npz -d /content/buddi/essentials/body_models/smplx -o SMPLX_NEUTRAL.npz
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/buddi/resolve/main/essentials/body_models/smplx/SMPLX_NEUTRAL.pkl -d /content/buddi/essentials/body_models/smplx -o SMPLX_NEUTRAL.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/buddi/resolve/main/essentials/body_models/smpla/SMPLA_NEUTRAL.pth -d /content/buddi/essentials/body_models/smpla -o SMPLA_NEUTRAL.pth

# %cd /content
# !pip install simple_romp==1.1.3 chumpy
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/SMPLer-X/resolve/main/SMPL_NEUTRAL.pkl -d /content/smpl_model_data -o SMPL_NEUTRAL.pkl
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/buddi/resolve/main/romp/J_regressor_extra.npy -d /content/smpl_model_data -o J_regressor_extra.npy
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/buddi/resolve/main/romp/J_regressor_h36m.npy -d /content/smpl_model_data -o J_regressor_h36m.npy
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/buddi/resolve/main/romp/smpl_kid_template.npy -d /content/smpl_model_data -o smpl_kid_template.npy
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/buddi/resolve/main/romp/smil_web.pkl -d /content/smpl_model_data/smil -o smil_web.pkl
# !mkdir /content/romp
# !romp.prepare_smpl -source_dir=/content/smpl_model_data --save_dir=/content/romp
# !bev.prepare_smil -source_dir=/content/smpl_model_data --save_dir=/content/romp
# !bev.prepare_smil -source_dir=/content/smpl_model_data
# !cp -r /root/.romp/. /content/smpl_model_data

In [ ]:
%cd /content/buddi
!python llib/methods/hhc_diffusion/evaluation/sample.py --exp-cfg essentials/buddi/buddi_unconditional.yaml --output-folder demo/diffusion/samples/ --checkpoint-name essentials/buddi/buddi_unconditional.pt --max-images-render=100 --num-samples 100 --max-t 1000 --skip-steps 10 --log-steps=100 --save-vis 

In [ ]:
%cd /content/buddi
!python llib/methods/hhcs_optimization/main.py --exp-cfg llib/methods/hhcs_optimization/configs/buddi_cond_bev_demo.yaml --exp-opts logging.base_folder=demo/optimization/buddi_cond_bev_demo datasets.train_names=['demo'] datasets.train_composition=[1.0] datasets.demo.original_data_folder=demo/data/FlickrCI3D_Signatures/demo datasets.demo.image_folder=images model.optimization.pretrained_diffusion_model_ckpt=essentials/buddi/buddi_cond_bev.pt model.optimization.pretrained_diffusion_model_cfg=essentials/buddi/buddi_cond_bev.yaml logging.run=fit_buddi_cond_bev_flickrci3ds